In [1]:
import pandas as pd

# Load the CSV
file_path = r"C:\Users\t14\Downloads\sales+transactions+dataset+weekly\Sales_Transactions_Dataset_Weekly.csv"
df = pd.read_csv(file_path)

# Show basic info
print("Shape:", df.shape)
df.head()


Shape: (811, 107)


,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,Normalized 42,Normalized 43,Normalized 44,Normalized 45,Normalized 46,Normalized 47,Normalized 48,Normalized 49,Normalized 50,Normalized 51
0,P1,11,12,10,8,13,12,14,21,6,...,0.06,0.22,0.28,0.39,0.50,0.00,0.22,0.17,0.11,0.39
1,P2,7,6,3,2,7,1,6,3,3,...,0.20,0.40,0.50,0.10,0.10,0.40,0.50,0.10,0.60,0.00
2,P3,7,11,8,9,10,8,7,13,12,...,0.27,1.00,0.18,0.18,0.36,0.45,1.00,0.45,0.45,0.36
3,P4,12,8,13,5,9,6,9,13,13,...,0.41,0.47,0.06,0.12,0.24,0.35,0.71,0.35,0.29,0.35
4,P5,8,5,13,11,6,7,9,14,9,...,0.27,0.53,0.27,0.60,0.20,0.20,0.13,0.53,0.33,0.40


In [5]:
# Drop duplicates
df_traditional = df.drop_duplicates()

# Identify weekly sales columns
weekly_cols = [col for col in df_traditional.columns if col.startswith('W') and col[1:].isdigit()]

# Convert to numeric (just in case)
df_traditional[weekly_cols] = df_traditional[weekly_cols].apply(pd.to_numeric, errors='coerce')

# Fill missing values with the column mean (basic traditional method)
df_traditional[weekly_cols] = df_traditional[weekly_cols].fillna(df_traditional[weekly_cols].mean())

# Optional: Normalize sales manually using Min-Max scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_traditional[weekly_cols] = scaler.fit_transform(df_traditional[weekly_cols])

# Show cleaned traditional data
df_traditional.head()


,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,Normalized 42,Normalized 43,Normalized 44,Normalized 45,Normalized 46,Normalized 47,Normalized 48,Normalized 49,Normalized 50,Normalized 51
0,P1,0.203704,0.226415,0.178571,0.135593,0.213115,0.230769,0.250000,0.338710,0.095238,...,0.06,0.22,0.28,0.39,0.50,0.00,0.22,0.17,0.11,0.39
1,P2,0.129630,0.113208,0.053571,0.033898,0.114754,0.019231,0.107143,0.048387,0.047619,...,0.20,0.40,0.50,0.10,0.10,0.40,0.50,0.10,0.60,0.00
2,P3,0.129630,0.207547,0.142857,0.152542,0.163934,0.153846,0.125000,0.209677,0.190476,...,0.27,1.00,0.18,0.18,0.36,0.45,1.00,0.45,0.45,0.36
3,P4,0.222222,0.150943,0.232143,0.084746,0.147541,0.115385,0.160714,0.209677,0.206349,...,0.41,0.47,0.06,0.12,0.24,0.35,0.71,0.35,0.29,0.35
4,P5,0.148148,0.094340,0.232143,0.186441,0.098361,0.134615,0.160714,0.225806,0.142857,...,0.27,0.53,0.27,0.60,0.20,0.20,0.13,0.53,0.33,0.40


In [3]:
!pip install pycaret

In [8]:
from pycaret.regression import *

# Copy original data and ensure weekly columns are numeric
df_ai = df.copy()
df_ai[weekly_cols] = df_ai[weekly_cols].apply(pd.to_numeric, errors='coerce')

# Create a Total_Sales column
df_ai['Total_Sales'] = df_ai[weekly_cols].sum(axis=1)

# Setup PyCaret (ignore Product_Code, MIN, MAX, and Normalized columns)
ignore_cols = ['Product_Code', 'MIN', 'MAX'] + [col for col in df.columns if 'Normalized' in col]
reg1 = setup(data=df_ai, target='Total_Sales', session_id=123,
             ignore_features=ignore_cols, normalize=True, verbose=False, html=False)
             # Replaced 'silent=True' with 'verbose=False' which is the correct parameter in PyCaret

# Run AutoML
best_model = compare_models()


                                    Model       MAE          MSE      RMSE  \
lr                      Linear Regression    0.0000       0.0000    0.0000   
ridge                    Ridge Regression    0.0545       0.0073    0.0846   
lar                Least Angle Regression    0.0000       0.0000    0.0000   
llar         Lasso Least Angle Regression    0.8958       1.2870    1.1263   
br                         Bayesian Ridge    0.0000       0.0000    0.0000   
par          Passive Aggressive Regressor    0.0474       0.0053    0.0715   
huber                     Huber Regressor    0.0005       0.0000    0.0008   
en                            Elastic Net    5.1390      44.6216    6.6306   
lasso                    Lasso Regression    6.2712     102.0731   10.0301   
et                  Extra Trees Regressor   17.6873    1131.6300   31.9934   
knn                 K Neighbors Regressor   21.9922    1350.0217   35.5981   
rf                Random Forest Regressor   21.1840    1521.3576

In [9]:
df_traditional.to_csv("cleaned_traditional.csv", index=False)
df_ai.to_csv("cleaned_ai.csv", index=False)
